#### Reading data from csv 

In [ ]:
sms_df.to_csv(r'/Users/annukajla/Desktop/Rwanda_sms_pred_demo.csv', index=False)
final_df=pd.read_csv(r"/Users/annukajla/Desktop/Rwanda_sms_pred_demo.csv")
df=final_df[:690000]  #taking a subset of dataset to run the model

Distribution of paid and not paid customers

In [ ]:
df['target'].value_counts(normalize=True)

In [ ]:
#drop target variable
X = df.drop(['target'], axis=1)

y =df['target']

### Optimal threshold

In [ ]:
def optimal_threshold(y, y_hat, beta):
    precision, recall, thresholds = precision_recall_curve(y, y_hat)
    nonzero_idx = np.where((recall != 0) & (precision != 0))  
    fscore = (beta * beta + 1) * precision[nonzero_idx] * recall[nonzero_idx] / \
             (beta * beta * precision[nonzero_idx] + recall[nonzero_idx])
    optimal_idx = np.argmax(fscore)
    return thresholds[optimal_idx]

def probabilities_to_binary(probabilities, threshold):
    return np.where(probabilities >= threshold, 1, 0)

### Function for confusion matrix

In [ ]:
def confusion_mat(y_test,y_pred):
    matrix = confusion_matrix(y_test, y_pred,normalize='all').round(2)
#     matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(7,5))
    ax = plt.subplot()
    sns.set(font_scale=1.5) # Adjust to fit
    sns.heatmap(matrix, annot=True, ax=ax, cmap="Greens", fmt="g");  
    class_names = ['not paid', 'paid']
    tick_marks = np.arange(len(class_names))
    tick_marks2 = tick_marks + 0.5
    
    # Labels, title and ticks
    label_font = {'size':'12'}  # Adjust to fit
    ax.set_xlabel('Predictions', fontdict=label_font);
    ax.set_ylabel('Actual', fontdict=label_font);
    plt.xticks(tick_marks, class_names, rotation=25)
    plt.yticks(tick_marks2, class_names, rotation=0)
    # plt.yticks([0.5,1.5], ['good prospects - ', 'bad prospects - '],rotation=0)


    # plt.xticks([0.5,1.5], ['| \n% of customers allowed \nby the CP', '| \n% of customers denied \nby the CP'])

    title_font = {'size':'21'}  # Adjust to fit
    # ax.set_title('Breakdown of current process against ground truth', fontdict=title_font);

    ax.tick_params(axis='both', which='major', labelsize=14)  # Adjust to fit
    plt.tight_layout()
    plt.show()
    # Build the plot
  

    print(classification_report(y_test, y_pred))
    

### model creation

In [ ]:
lgbm_binary_clf = lgb.LGBMClassifier(
  objective="binary",
    metric='',
    max_depth= -1,
    silent=True
)

### Selecting Hyperparameters

In [ ]:
hyperparameters_grid_space =    {
    'n_estimators': np.arange(50,550,50),
    'boosting_type': ['gbdt', 'goss', 'dart'],
    'num_leaves': list(range(100, 150,3)),
    'learning_rate': list(np.logspace(np.log10(0.005), np.log10(0.5), base = 10, num = 500)),
    'subsample_for_bin': list(range(20000, 300000, 20000)),
    'min_child_samples': list(range(20, 500, 5)),
    'reg_alpha': list(np.linspace(0, 1)),
    'reg_lambda': list(np.linspace(0, 1)),
    'colsample_bytree': list(np.linspace(0.6, 1, 10)),
    'subsample': list(np.linspace(0.5, 1, 100)),
    'scale_pos_weight': [1, 1.3, 1.5]
}

# setup CV parameters
n_jobs = -1
beta = 0.5
seed = 42
verbose = 1
cv = 4
test_size=0.25
n_combinations = 10  

#stratified_kfolds to do cross validation
stratified_kfolds = StratifiedKFold(n_splits=cv, random_state=seed, shuffle=True)


ftwo_scorer = make_scorer(fbeta_score, beta=beta)    # using fbeta_score as metrics

random_cv = RandomizedSearchCV(estimator=lgbm_binary_clf,
                               n_iter=n_combinations,
                               n_jobs=n_jobs,
                               param_distributions=hyperparameters_grid_space,
                               random_state=seed,
                               scoring=ftwo_scorer,
                               cv = stratified_kfolds,
                               verbose=verbose)

### Nested Cross Validation

In [ ]:
#def nested_cv(train_features,train_target,random_cv,model):
outer_cv_params = []
outer_cv_scores = []
outer_cv_ots = []

for train_idx, test_idx in stratified_kfolds.split(X, y):
    X_train_cv = X.iloc[train_idx]
    X_test_cv = X.iloc[test_idx]
    y_train_cv = y.iloc[train_idx]
    y_test_cv = y.iloc[test_idx]
    X_resampled, y_resampled = os.fit_resample(X_train_cv, y_train_cv)
    ftwo_scorer = make_scorer(fbeta_score, beta=beta)
    # cv to tuning
    random_cv.fit(X_resampled, y_resampled)
    outer_cv_params.append(random_cv.best_params_)
#     print(y_resampled.value_counts())
    # train model with best params
    lgbm_binary_clf.set_params(**random_cv.best_params_)
    lgbm_binary_clf.fit(X_resampled, y_resampled)

    # predict on testset
    y_hat = lgbm_binary_clf.predict(X_test_cv)
    target_test_predicted_proba = lgbm_binary_clf.predict_proba(X_test_cv)[:, 1]
    
    # tune threshold
    ot = optimal_threshold(y_test_cv, target_test_predicted_proba, beta=beta)
    outer_cv_ots.append(ot)
    
    # convert to probabilities and score
    y_test_pred = probabilities_to_binary(target_test_predicted_proba, ot)
    score = fbeta_score(y_test_cv, y_test_pred,beta=beta).round(2)
    outer_cv_scores.append(score)
   # F1_score = f1_score(y_test_cv,y_hat,average='weighted')
    #outer_cv_scores.append(F1_score)
    print(outer_cv_scores,ot)
    

### Fitting the model

In [ ]:
random_cv.fit(X, y)

In [ ]:
best_params = random_cv.best_params_
lgbm_binary_clf.set_params(**best_params)  
lgbm_binary_clf.fit(X,y) 

In [ ]:
y_pred = lgbm_binary_clf.predict_proba(X)[:,1]
print(y_pred)

# tune threshold
optimal_th = optimal_threshold(y, y_pred, beta=0.5)
predicted_classes = probabilities_to_binary(y_pred, optimal_th)
# score
total_score = fbeta_score(y, predicted_classes,beta=beta)

print("total score",round(total_score, 2))
pd.value_counts(predicted_classes, normalize=True)

### Confussion Matrix

In [ ]:
confusion_mat(y,predicted_classes)

### Feature Importance

In [ ]:
shap.initjs()
k_explainer = shap.TreeExplainer(lgbm_binary_clf)

shap_values = k_explainer.shap_values(X)
shap.summary_plot(shap_values[1], X)

### Error Analysis

In [ ]:
result = pd.concat([X, y], axis=1)
result['predicted_target'] = predicted_classes.tolist()
result['predicted_target_probabilities']=y_pred.tolist()

In [ ]:
result.loc[(result["target"] == 1) & (result["predicted_target"] == 1), "TP/FN"] = 'TP'
result.loc[(result["target"] == 0) & (result["predicted_target"] == 0), "TP/FN"] = 'TN'
result.loc[(result["target"] == 0) & (result["predicted_target"] == 1), "TP/FN"] = 'FP'
result.loc[(result["target"] == 1) & (result["predicted_target"] == 0), "TP/FN"] = 'FN'
result.head()

Wrong predictions  by the model (FP/FN)

In [ ]:
result['TP/FN'].value_counts().to_frame()

False Negatives

In [ ]:
False_neg=result[result['TP/FN']=='FN']
False_neg=False_neg.sort_values(by=['predicted_target_probabilities'], ascending=False)
print("Total FN cases",len(False_neg))
False_neg.head(15)

False Positives

In [ ]:
False_pos=result[result['TP/FN']=='FP']
False_pos=False_pos.sort_values(by=['predicted_target_probabilities'], ascending=False)
print("Total FP cases",len(False_pos))
False_pos.head(15)

In [ ]:
df_error_analysis=result[[ 'daily_rate',
       'utilisation_rate', 'balance_left', 'age', 'days_customer_in_bbox',
        'Total_no_of_payments','TP/FN']].copy()

In [ ]:
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

fig = make_subplots(
    rows=7, cols=1,
#     subplot_titles=("Plot 1", "Plot 2", "Plot 3", "Plot 4","Plot 5","Plot 6","Plot 7","Plot 8")
)
counter=0
#y_data=df_error_analysis[df_error_analysis.columns]
colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)','rgba(255, 65, 54, 0.5)']
x_data = ['TN ', 'TP',
          'FP', 'FN',]
for i in range(1,8):
    if counter < 8:
        fig.add_trace(go.Box(y=df_error_analysis[df_error_analysis.columns[counter]],
                x=df_error_analysis['TP/FN'],quartilemethod="linear"),
                  row=counter+1, col=1)


        fig.update_yaxes(title_text=df_error_analysis.columns[counter],row=counter+1, col=1)
        counter+=1
#fig.update_traces(boxpoints='all', jitter=0)
fig.update_layout(height=1600, width=1200,showlegend=False,
                  title_text="Multiple Subplots with Titles")

fig.show()

Summary:

    Total 3 day reminder msgs sent till date:7.4M
    Total 3 day reminder msgs sent in year 2022: 1.2M
    1) Messages sent to invalid phone number:
        i)total 3.8 million msgs sent to invalid phone numbers till date'
        ii)500K msgs in 2022
        iii)100K msgs for 3 day reminder in year 2022
        
    2) Messages sent to repossessed customers
        i)73K msgs sent to repossessed customers till date
        ii)40K msgs sent to repossessed customers for year 2022

    4) Did not find any duplication of messages for rwanda
    
    5)dataset after cleaning =1.6 Million for year 2022:
        paid:65%
        not paid:35%